In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch
import pytorch_lightning as pl
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.logging import TensorBoardLogger
import os
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
data_path = Path('./data')

train = pd.read_csv(data_path/'train.csv')

test = pd.read_csv(data_path / 'data_for_test.csv')

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')


In [4]:

label_enc = LabelEncoder()

train = train.sort_values(['patient_id', 'key'])

features =  ['gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5',
       'x6', 'xx1', 'xx2','xx3','xx4','xx5']

cat_features = ['x3']

cont_features = [i for i in features if i not in cat_features]

scaler = StandardScaler()

train[cont_features] = scaler.fit_transform(train[cont_features].values)

train[cat_features] = label_enc.fit_transform(train[cat_features].values)

train.x3.unique()

val_patients = np.random.choice(train.patient_id.unique(), size=500, replace=False)

print(train.shape[0])
val = train[train.patient_id.isin(val_patients)]
train = train[~train.patient_id.isin(val_patients)]
print(val.shape[0]+ train.shape[0])



1348470
1348470


/data/nithish/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
class HeartDataset(Dataset):
    
    def __init__(self, dataframe, ts_features, cont_features, cat_features, target_columns=['y_mean_MAP', 'y_mean_HR']):
        self.dataframe = dataframe.reset_index(drop=True)
        self.ts_features = ts_features
        self.cont_features = cont_features
        self.cat_features = cat_features
        self.target_columns = target_columns
        
    def __len__(self):
        return self.dataframe.key.nunique()
    
    def __getitem__(self,idx):

        timeseries = self.dataframe.iloc[30*idx : 30*(idx+1)][self.ts_features].values
        timeseries = timeseries.T  # C_in x 30 (timestamps)
        
        cont_features = self.dataframe.loc[30*idx,self.cont_features].values
        cat_features = self.dataframe.loc[30*idx,self.cat_features].values
        
        if self.target_columns:
            target = self.dataframe.iloc[30*idx][self.target_columns].values
            return [timeseries, cont_features, cat_features, target]
        else:
            return [timeseries, cont_features, cat_features]
            

In [7]:
ts_features = ['xx1', 'xx2', 'xx3', 'xx4', 'xx5']
cat_features = ['x3']

cont_features = ['gender', 'age', 'x1', 'x2', 'x4', 'x5', 'x6']

target_columns = ['y_mean_MAP', 'y_mean_HR']


dataset_train = HeartDataset(train,ts_features, cont_features, cat_features, target_columns)
dataset_val = HeartDataset(val,ts_features, cont_features,cat_features , target_columns)


In [1]:
import torch.nn as nn

In [ ]:
nn.MultiheadAttention()

In [8]:
def custom_collate(batch):
    ts = np.stack([item[0] for item in batch])
    cont_features = np.stack([item[1] for item in batch])
    cat_features = np.stack([item[2] for item in batch])
    
    
    
    ts = torch.from_numpy(ts).float()
    cont_features = torch.from_numpy(cont_features.astype(float)).float()
    cat_features = torch.from_numpy(cat_features.astype(float)).long()
    
    
    # use gpu
    
    ts = ts.to(device)
    cont_features = cont_features.to(device)
    cat_features = cat_features.to(device)
    
    
    try: 
        target = np.stack([item[3] for item in batch])
        target = torch.from_numpy(target.astype(float)).float()
        target = target.to(device)
        return [ts, cont_features, cat_features, target]

    except:
        return [ts, cont_features, cat_features]

def r2_squared_loss_function(preds , target):
    '''
    PREDICTIONS --> TARGET ORDER IMPORTANT
    
    '''
    
    mean_target = torch.mean(target,0)

    mean_mse = torch.mean((target-mean_target)**2,0)
    pred_mse = torch.mean((preds-target)**2,0)
    
    div = 1 - pred_mse/(mean_mse+10e-6)
    
    mean_r2_squared = torch.mean(div)
    
    return -mean_r2_squared


In [9]:
# batch_size =  128
# dataloader_train = DataLoader(dataset_train, batch_size=batch_size,collate_fn=custom_collate)
# dataloader_val = DataLoader(dataset_val, batch_size=batch_size,collate_fn=custom_collate)

In [10]:



class CoolSystem(pl.LightningModule):

    def __init__(self, batch_size):
        super(CoolSystem, self).__init__()
        # not the best model...
        self.fc1 = nn.Linear(167,64)
        self.fc2 = nn.Linear(64,2)
        self.emb = nn.Embedding(5, 10)
        self.flat1 = nn.Flatten()
        self.batch_size = batch_size

    def forward(self, ts, cont, cat):
        ts = self.flat1(ts)
        cat = self.emb(cat).squeeze(1)
        merged = torch.cat([ts,cont,cat],axis=1)
        merged = self.fc1(merged)
        merged = F.relu(merged)
        merged = self.fc2(merged)
#         merged = F.relu(merged)
#         merged = self.fc3(merged)
        
        return merged

    def training_step(self, batch, batch_idx):
        # REQUIRED
        ts, cont, cat, y = batch
        y_hat = self.forward(ts, cont, cat)
        loss = r2_squared_loss_function(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        # OPTIONAL
        ts, cont, cat, y = batch
        y_hat = self.forward(ts, cont, cat)
        loss = r2_squared_loss_function(y_hat, y)
        return {'val_loss': loss}

    def validation_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=0.03)

    @pl.data_loader
    def train_dataloader(self):
        # REQUIRED
        return DataLoader(dataset_train, batch_size=self.batch_size,
                          collate_fn=custom_collate,shuffle=True)

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(dataset_val, batch_size=self.batch_size,
                          collate_fn=custom_collate, shuffle=True)


# GRU

In [11]:
# dataloader_train = DataLoader(dataset_train, batch_size=32,
#                           collate_fn=custom_collate,shuffle=True)
# batch = next(iter(dataloader_train))

# ts, cont, cat, y = batch

# emb_layer = nn.Embedding(5, 10)

# cat = emb_layer(cat).squeeze(1)

# cat.shape

# ts.shape, cont.shape, cat.shape

In [12]:
# gru1 = nn.GRU(input_size=5, hidden_size=64, batch_first=True,
#              num_layers=1)



# gru2 = nn.GRU(input_size=64, hidden_size=32, batch_first=True,
#              num_layers=1)



In [13]:



class SimpleGRU(pl.LightningModule):

    def __init__(self, hparams):
        super(SimpleGRU, self).__init__()
        
        # batch_size
        self.hparams = hparams
        
        #emb layer 
        self.emb = nn.Embedding(5,10)
        # layers
        # conv layers
        self.gru1 = nn.GRU(input_size=5, hidden_size=64, batch_first=True,
             num_layers=3)
 
        self.gru2 = nn.GRU(input_size=64, hidden_size=32, batch_first=True,
             num_layers=1)

        self.relu_merged = nn.ReLU()
        
        self.fc1 = nn.Linear(in_features=49, out_features=32)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(in_features=16, out_features=2)

    def forward(self, ts, cont, cat):
        
        # ts into GRU
        o1,o2 = self.gru1(ts.transpose(1,2))
        o3,o4 = self.gru2(o1)
        o4 = o4.squeeze(0)

        # remove cat dimen
        cat = self.emb(cat).squeeze(1)
        
        # merge after embedding
        merged = torch.cat([o4,cont,cat],axis=1)
        merged = self.relu_merged(merged)
        
        merged = self.fc1(merged)
        merged = self.relu1(merged)
        merged = self.fc2(merged)
        merged = self.relu2(merged)
        merged = self.fc3(merged)

        
        return merged

    def training_step(self, batch, batch_idx):
        # REQUIRED
        ts, cont, cat, y = batch
        y_hat = self.forward(ts, cont, cat)
        loss = r2_squared_loss_function(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        # OPTIONAL
        ts, cont, cat, y = batch
        y_hat = self.forward(ts, cont, cat)
        loss = r2_squared_loss_function(y_hat, y)
        return {'val_loss': loss}

    def validation_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

    @pl.data_loader
    def train_dataloader(self):
        # REQUIRED
        return DataLoader(dataset_train, batch_size=self.hparams.batch_size,
                          collate_fn=custom_collate,shuffle=True)

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(dataset_val, batch_size=self.hparams.batch_size,
                          collate_fn=custom_collate, shuffle=True)


In [14]:
from argparse import Namespace


In [ ]:
name = 'gru3relus'

logger = TensorBoardLogger(
                save_dir='ts-logs',
                name = name

            )

early_stopping = EarlyStopping('val_loss',patience=5)
checkpoint_callback = ModelCheckpoint(filepath=f'saved_models/{name}')

trainer = pl.Trainer(logger=logger,early_stop_callback=early_stopping,checkpoint_callback=checkpoint_callback)

hparams = {'batch_size':4096,
          'lr':0.1}

hparams = Namespace(**hparams)

simplegru = SimpleGRU(hparams)
# model = model.cuda()

simplegru = simplegru.to(device)

trainer.fit(simplegru)

INFO:root:
          Name       Type Params
0          emb  Embedding   50  
1         gru1        GRU   63 K
2         gru2        GRU    9 K
3  relu_merged       ReLU    0  
4          fc1     Linear    1 K
5        relu1       ReLU    0  
6          fc2     Linear  528  
7        relu2       ReLU    0  
8          fc3     Linear   34  
                                                                         

Epoch 1:   0%|          | 0/12 [00:00<?, ?batch/s]

Epoch 13:  42%|████▏     | 5/12 [00:52<01:10, 10.08s/batch, batch_idx=4, loss=1.177, v_num=0] 

In [21]:



class SimpleCNN(pl.LightningModule):

    def __init__(self, hparams):
        super(SimpleCNN, self).__init__()
        
        # batch_size
        self.hparams = hparams
        self.batch_size = self.hparams.batch_size  # for dataloaders
        
        #emb layer 
        self.emb = nn.Embedding(5,10)
        # layers
        # conv layers
        self.cnn1 = nn.Conv1d(in_channels=5, out_channels=128, kernel_size=3, dilation=1,)
        self.relu1 = nn.ReLU()
#         self.maxpool1 = nn.MaxPool1d(kernel_size=3)
        
        self.cnn2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.relu2 = nn.ReLU()
#         self.maxpool2 = nn.MaxPool1d(kernel_size=3)
        
        self.cnn3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3)
        self.relu3 = nn.ReLU()
#         self.maxpool3 = nn.MaxPool1d(kernel_size=3)
        
        self.cnn_flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=785, out_features=32)
        self.relu_combined = nn.ReLU()
        self.fc2 = nn.Linear(in_features=32, out_features=2)

    def forward(self, ts, cont, cat):
        ts = self.relu1(self.cnn1(ts)) # first conv
        ts = self.relu2(self.cnn2(ts))
        ts = self.relu3(self.cnn3(ts))
        ts = self.cnn_flatten(ts)
        
        cat = self.emb(cat).squeeze(1)
        merged = torch.cat([ts,cont,cat],axis=1)
        merged = self.fc1(merged)
        merged = self.relu_combined(merged)
        merged = self.fc2(merged)
#         merged = F.relu(merged)
#         merged = self.fc3(merged)
        
        return merged

    def training_step(self, batch, batch_idx):
        # REQUIRED
        ts, cont, cat, y = batch
        y_hat = self.forward(ts, cont, cat)
        loss = r2_squared_loss_function(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        # OPTIONAL
        ts, cont, cat, y = batch
        y_hat = self.forward(ts, cont, cat)
        loss = r2_squared_loss_function(y_hat, y)
        return {'val_loss': loss}

    def validation_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

    @pl.data_loader
    def train_dataloader(self):
        # REQUIRED
        return DataLoader(dataset_train, batch_size=self.batch_size,
                          collate_fn=custom_collate,shuffle=True)

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(dataset_val, batch_size=self.batch_size,
                          collate_fn=custom_collate, shuffle=True)


In [112]:
name = 'no_maxpool'

logger = TensorBoardLogger(
                save_dir='ts-logs',
                name = name

            )

early_stopping = EarlyStopping('val_loss',patience=5)
checkpoint_callback = ModelCheckpoint(filepath=f'saved_models/{name}')

trainer = pl.Trainer(logger=logger,early_stop_callback=early_stopping,checkpoint_callback=checkpoint_callback)

hparams = {'batch_size':1024,
          'lr':3e-3}

hparams = Namespace(**hparams)

simplecnn = SimpleCNN(hparams)
# model = model.cuda()

simplecnn = simplecnn.to(device)

trainer.fit(simplecnn)

In [33]:
whole_val_metric(model, dataset_val)

tensor(-0.8792, grad_fn=<NegBackward>)


In [32]:
def whole_val_metric(model, dataset_val):
    model.eval()
    dl = DataLoader(dataset_val, batch_size=len(dataset_val),collate_fn=custom_collate)
    ts, cont, cat, y = next(iter(dl))
    y_hat = model(ts, cont, cat)
    print(r2_squared_loss_function(y_hat, y))

In [ ]:
def get_num_parameters(model):
    s = 0
    for param in model.parameters():
        s += param.numel()
    return s

### Test predictions